In [6]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

def calculate_metrics(true_values, pred_values):
    mse = round(mean_squared_error(true_values, pred_values),3)
    mae = round(mean_absolute_error(true_values, pred_values),3)
    r_score = round(r2_score(true_values, pred_values),3)

    return {"mse": mse,
            "mae": mae,
            "r_score": r_score,}

In [7]:
import h2o
from h2o.automl import H2OAutoML

# Start the H2O cluster (locally)
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,3 hours 53 mins
H2O_cluster_timezone:,Europe/Kiev
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,1 month and 9 days
H2O_cluster_name:,H2O_from_python_38066_gx568w
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.481 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [25]:
import os
import sys
sys.path.insert(0, os.path.dirname('C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part'))

import pandas as pd

from ml_part.random_forest.data_prep.preparation import DataPreparation
from ml_part.random_forest.train import RFTrain

CSV_PATH = r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\remained_features_25.01.csv'
smiles_to_subgroup_filepath = r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\smiles_to_subgroup.pkl'
smiles_filepath = r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\smiles_to_index.pkl'

dataPreparation = DataPreparation(path_to_data=CSV_PATH,
                                  smiles_to_subgroup_pickle_file=smiles_to_subgroup_filepath,
                                  smiles_to_index_pickle_file=smiles_filepath)

unimportant_features_to_drop = ['nFaRing', 'nHRing', 'f_atom_fraction', 'tpsa+f', 'nFaHRing',
       'nFAHRing', 'naRing', 'cis/trans', 'dipole_moment', 'nFARing',
       'distance_between_atoms_in_f_group_centers', 'nF', 'PBF', 'naHRing',
       'RPCS', 'GeomShapeIndex', 'nFHRing', 'mol_num_cycles']

X, y = dataPreparation.prepare_data_for_RF(is_pKa=False,
                                           subgroup_type="Carboxylic acid",
                                           use_mandatory_features=True,
                                           is_remove_outliers=True,
                                           is_remove_nan=False,
                                           outliers_features_to_skip=unimportant_features_to_drop)

features_to_drop = []
for feature_name in X.columns:
    if 'angle' in feature_name or feature_name in unimportant_features_to_drop:
        features_to_drop.append(feature_name)

X = X.drop(features_to_drop, axis=1)

rf_train = RFTrain(X=X, 
                   y=y,
                   smiles_filepath=smiles_filepath,
                   is_pKa=False,
                   k_folds=2)

y_train = rf_train.y_train
X_train = rf_train.X_train

y_test = rf_train.y_test
X_test = rf_train.X_test

train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

print(len(train_df), len(test_df))

train = h2o.H2OFrame(train_df)
test = h2o.H2OFrame(test_df)

True
55
['dipole_moment', 'FPSA3', 'nHRing', 'avg_atoms_in_cycle', 'angle_X1X2R2', 'PNSA5', 'nN', 'angle_R2X2R1', 'nF', 'nFRing', 'tpsa+f', 'f_freedom', 'f_atom_fraction', 'mol_num_cycles', 'nO', 'nFAHRing', 'angle_R1X1R2', 'distance_between_atoms_in_f_group_centers', 'nC', 'PBF', 'nAHRing', 'mol_volume', 'nFaRing', 'f_to_fg', 'GeomShapeIndex', 'cis/trans', 'PPSA5', 'nFHRing', 'angle_X2X1R1', 'nFaHRing', 'RPCS', 'naRing', 'nFARing', 'dihedral_angle', 'naHRing', 'chirality', 'nARing', 'TASA', 'pKa', 'logP']
FPSA3 [40]
PPSA5 [39, 40]
logP [48]
Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54],
      dtype='int64')
Remains rows:51, amount of features: 40
41 10
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████

In [17]:
x = train.columns
y = "logP"
x.remove(y)

aml = H2OAutoML(seed=1, max_runtime_secs=1800)
# aml.train(x=x, y=y, training_frame=train, leaderboard_frame=test, fold_column="fold_id")
aml.train(x=x, y=y, training_frame=train, fold_column="fold_id")

# View the AutoML Leaderboard
lb = aml.leaderboard
print(lb.head(rows=lb.nrows))

AutoML progress: |█
17:53:21.234: Fold column fold_id will be used for cross-validation. nfolds parameter will be ignored.
17:53:21.235: AutoML: XGBoost is not available; skipping it.
17:53:21.237: _train param, Dropping bad and constant columns: [nN, nAHRing, nO]
17:53:21.303: _train param, Dropping bad and constant columns: [nN, nAHRing, nO]
17:53:21.303: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 47.0.
17:53:21.306: _train param, Dropping bad and constant columns: [nN, nAHRing, nO]
17:53:21.418: _train param, Dropping bad and constant columns: [nN, nAHRing, nO]
17:53:21.547: _train param, Dropping bad and constant columns: [nN, nAHRing, nO]
17:53:21.639: _train param, Dropping bad and constant columns: [nN, nAHRing, nO]
17:53:21.735: _train param, Dropping unused columns: [nN, nAHRing, nO]
17:53:21.845: _train param, Dropping unused columns: [nN, nAHRing, nO]
17:53:21.954: _train param, Dropping

In [20]:
best_model = h2o.get_model('GBM_grid_1_AutoML_4_20240129_175321_model_276')
preds = best_model.predict(test)
true = test.as_data_frame()['logP']
predicted_values = preds.as_data_frame()['predict']

calculate_metrics(true, predicted_values)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


c:\work\DrugDiscovery\drug-discovery-venv\Lib\site-packages\h2o\frame.py:1979: H2ODependencyWarning: converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above).
  warnings.warn("converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
c:\work\DrugDiscovery\drug-discovery-venv\Lib\site-packages\h2o\frame.py:1979: H2ODependencyWarning: converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above).
  warnings.warn("converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


{'mse': 0.057, 'mae': 0.184, 'r_score': 0.612}

In [21]:
best_model = aml.get_best_model()
preds = best_model.predict(test)
true = test.as_data_frame()['logP']
predicted_values = preds.as_data_frame()['predict']

calculate_metrics(true, predicted_values)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


c:\work\DrugDiscovery\drug-discovery-venv\Lib\site-packages\h2o\frame.py:1979: H2ODependencyWarning: converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above).
  warnings.warn("converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
c:\work\DrugDiscovery\drug-discovery-venv\Lib\site-packages\h2o\frame.py:1979: H2ODependencyWarning: converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above).
  warnings.warn("converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


{'mse': 0.055, 'mae': 0.162, 'r_score': 0.629}

In [22]:
best_model

key,value
Stacking strategy,cross_validation
Number of base models (used / total),3/5
# GBM base models (used / total),1/1
# DeepLearning base models (used / total),1/1
# GLM base models (used / total),1/1
# DRF base models (used / total),0/2
Metalearner algorithm,GLM
Metalearner fold assignment scheme,AUTO
Metalearner nfolds,0
Metalearner fold_column,fold_id


In [16]:
MODEL_SAVE_PATH = r'C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part\h2o_model\25.01.24_features_3_subgroups\logP_models'

model_path = h2o.save_model(model=best_model, path=os.path.join(MODEL_SAVE_PATH, 'primary_amine_subgroup_all_molecules_top15_features'), force=True)

model_path

'C:\\work\\DrugDiscovery\\main_git\\XAI_Chem\\ml_part\\h2o_model\\25.01.24_features_3_subgroups\\logP_models\\primary_amine_subgroup_all_molecules_top15_features\\DeepLearning_grid_2_AutoML_3_20240129_172436_model_10'

In [28]:
model_path = r'C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part\h2o_model\models\25.01.24_features\logP\Only_mol_with_angles_without_outliers(except_dipole)_with_angles(as features)\DeepLearning_grid_1_AutoML_2_20240127_15921_model_83'

test_model = h2o.load_model(model_path)

test_model

Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_1_AutoML_2_20240127_15921_model_83


Status of Neuron Layers: predicting logP, regression, gaussian distribution, Quadratic loss, 721 weights/biases, 15,6 KB, 890 890 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate               rate_rms                momentum    mean_weight           weight_rms          mean_bias              bias_rms
--  -------  -------  ----------------  ---------  ----  ----  ----------------------  ----------------------  ----------  --------------------  ------------------  ---------------------  -----------------------
    1        34       Input             0.0
    2        20       RectifierDropout  40.0       0.0   0.0   0.0005347729694489424   0.00043007126078009605  0.0         0.005370978544413126  0.2091236114501953  -0.01838975090675744   0.15310299396514893
    3        1        Linear                       0.0   0.0   0.00010892761010836694  5.459638487081975e-05   0.0         0.04597926698625088   0.3017078638076782  0.0032811631519775344  1.0971281125650402e-154

ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 0.008251688509692242
RMSE: 0.0908388050873207
MAE: 0.062210524066970634
RMSLE: 0.037665745594135415
Mean Residual Deviance: 0.008251688509692242

ModelMetricsRegression: deeplearning
** Reported on cross-validation data. **

MSE: 0.12133309752662577
RMSE: 0.34832900758711693
MAE: 0.2543856307985424
RMSLE: 0.12692739866933195
Mean Residual Deviance: 0.12133309752662577

Cross-Validation Metrics Summary: 
                        mean      sd         cv_1_valid    cv_2_valid
----------------------  --------  ---------  ------------  ------------
mae                     0.254001  0.0161379  0.24259       0.265412
mean_residual_deviance  0.120559  0.0324599  0.0976068     0.143512
mse                     0.120559  0.0324599  0.0976068     0.143512
r2                      0.614486  0.0718267  0.665275      0.563696
residual_deviance       0.120559  0.0324599  0.0976068     0.143512
rmse                    0.345625  0.0469582  0.312421      0.37883
rmsle                   0.125744  0.0191793  0.112183      0.139306

Scoring History: 
    timestamp            duration           training_speed    epochs    iterations    samples    training_rmse    training_deviance    training_mae    training_r2
--  -------------------  -----------------  ----------------  --------  ------------  ---------  ---------------  -------------------  --------------  -------------
    2024-01-27 02:09:31  0.000 sec                            0         0             0          nan              nan                  nan             nan
    2024-01-27 02:09:31  10 min  2.493 sec  445000 obs/sec    10        1             890        0.610918         0.373221             0.447107        -0.188484
    2024-01-27 02:09:33  10 min  3.964 sec  605224 obs/sec    10010     1001          890890     0.0908388        0.00825169           0.0622105       0.973723

Variable Importances: 
variable                                   relative_importance    scaled_importance    percentage
-----------------------------------------  ---------------------  -------------------  --------------------
nN                                         1.0                    1.0                  0.047972542730959404
mol_volume                                 0.9709323048591614     0.9709323048591614   0.04657809148372502
PPSA5                                      0.8813123106956482     0.8813123106956482   0.042278792484167556
cis/trans                                  0.8379440903663635     0.8379440903663635   0.04019830868125528
PNSA5                                      0.762306272983551      0.762306272983551    0.03656977025478181
tpsa+f                                     0.7456972599029541     0.7456972599029541   0.03577299366505381
angle_R1X1R2                   